Given two strings s and t of lengths m and n respectively, return the minimum window substring of s such that every character in t (including duplicates) is included in the window. If there is no such substring, return the empty string "".

The testcases will be generated such that the answer is unique.

 

Example 1:

Input: s = "ADOBECODEBANC", t = "ABC"
Output: "BANC"
Explanation: The minimum window substring "BANC" includes 'A', 'B', and 'C' from string t.
Example 2:

Input: s = "a", t = "a"
Output: "a"
Explanation: The entire string s is the minimum window.
Example 3:

Input: s = "a", t = "aa"
Output: ""
Explanation: Both 'a's from t must be included in the window.
Since the largest window of s only has one 'a', return empty string.
 

Constraints:

m == s.length
n == t.length
1 <= m, n <= 105
s and t consist of uppercase and lowercase English letters.
 

Follow up: Could you find an algorithm that runs in O(m + n) time?

- start from each index, for loop 
- find the window which satisfies the condition
- track the minimum
- return the minumum
- tc - O(m * n)
- sc - O(2 * n).  #len(t)

In [ ]:
# here intution is to use sliding window, since we founc the magic word "substring"
# increase until it satisfies the condition
# decrease until it not violate the condition.

from collections import Counter 

class Solution:
    def minWindow(self, s: str, t: str) -> str:
        start = 0
        countet_t = Counter(t)
        counter_s = {s[0]: 1}
        m,n = len(s), len(t)
        mini = 1e9

        # create the first window.
        for end in range(1, m):
            if s[end] in countet_t:
                if s[end] not in counter_s:
                    counter_s[s[end]] = 1
                else:
                    #its presnt, but still we do need to match the frequnecies,
                    if counter_s[s[end]] < countet_t[s[end]]:
                        # add them
                        counter_s[s[end]] += 1
            if counter_s == countet_t:
                mini = min(mini, end - start + 1)
                break 
        
        # so now we found our first window, where the elements are present meeting the condition.

        for end in range(end+1, m):
            # if the current ele is the t, but not the start of the window.
            # we cant remove them, since we dont know we will get them or not.
            # So start strinking only when we find the element which is == s[start].
            if s[end] in countet_t:
                # this can be the first element, or not
                if s[start] == s[end]:
                    # add the end value to the window and remove the starting unwanted charcters.
                    counter_s[s[end]] += 1
                    # strink until the condition is kept.
                    # the starting element might be in the t.
                    con = True
                    while con:
                        if s[start] in countet_t:
                            # we need to check removing this wont affect the condition.
                            counter_s[s[start]] -= 1
                            if counter_s[s[start]] ==0:
                                del counter_s[s[start]]

                            if counter_s == countet_t:
                                start += 1
                            else:
                                counter_s[s[start]] = counter_s.get(s[start], 0) + 1
                                con = False
                        else:
                            start += 1
                else:
                    # keep it might be useful in the tracking.
                    counter_s[s[end]] += 1
            
            # when the frequency of the current charcter is matched 
            if counter_s == countet_t:
                mini = min(mini, end - start + 1)
        return mini

In [9]:
Solution().minWindow(s = "ADOBECODEBANC", t = "ABC")

6

In [10]:
# the above fails teriblly


In [ ]:
from collections import Counter

class Solution:
    def minWindow(self, s: str, t: str) -> str:
        if not t or not s:
            return ""

        # Count characters in t
        t_count = Counter(t)
        window_count = {}

        have = 0                # Number of characters that meet the required count
        need = len(t_count)     # Total unique characters we need to match

        res = [-1, -1]          # To store the start and end of the minimum window
        res_len = float("inf")  # Initialize with infinity
        left = 0                # Left pointer of the window

        # Expand the window with right pointer
        for right in range(len(s)):
            char = s[right]
            window_count[char] = window_count.get(char, 0) + 1

            # Count only if character is in t and required frequency met
            if char in t_count and window_count[char] == t_count[char]:
                have += 1

            # Contract the window from the left
            while have == need:
                # Update result if it's smaller
                if (right - left + 1) < res_len:
                    res = [left, right]
                    res_len = right - left + 1

                # Shrink the window
                window_count[s[left]] -= 1
                if s[left] in t_count and window_count[s[left]] < t_count[s[left]]:
                    have -= 1
                left += 1

        l, r = res
        return s[l:r+1] if res_len != float("inf") else ""


 Initial Setup:
t_count: {A:1, B:1, C:1}

window_count: {}

have = 0, need = 3

left = 0, res = [-1, -1], res_len = inf



| Right | Char | window\_count                  | Have | Action                                              | Left | Window | Valid? | Update `res`?        |
| ----- | ---- | ------------------------------ | ---- | --------------------------------------------------- | ---- | ------ | ------ | -------------------- |
| 0     | A    | {A:1}                          | 1    | A is needed, count matches                          | 0    | A      | ❌      |                      |
| 1     | D    | {A:1, D:1}                     | 1    | D not needed, skip                                  | 0    | AD     | ❌      |                      |
| 2     | O    | {A:1, D:1, O:1}                | 1    | O not needed, skip                                  | 0    | ADO    | ❌      |                      |
| 3     | B    | {A:1, D:1, O:1, B:1}           | 2    | B is needed, count matches                          | 0    | ADOB   | ❌      |                      |
| 4     | E    | {A:1, D:1, O:1, B:1, E:1}      | 2    | E not needed, skip                                  | 0    | ADOBE  | ❌      |                      |
| 5     | C    | {A:1, D:1, O:1, B:1, E:1, C:1} | 3    | C is needed, count matches → all needed chars found | 0    | ADOBEC | ✅      | res = \[0,5] (len 6) |
|       |      |                                |      | Try shrinking                                       | 1    | DOBEC  | ✅      | res = \[1,5] (len 5) |
|       |      |                                |      | Remove D                                            | 2    | OBEC   | ✅      | res = \[2,5] (len 4) |
|       |      |                                |      | Remove O                                            | 3    | BEC    | ✅      | res = \[3,5] (len 3) |
|       |      |                                |      | Remove B → have = 2                                 | 4    | EC     | ❌      |                      |
| 6     | O    | {..., O:2}                     | 2    | O not needed                                        | 4    | ECO    | ❌      |                      |
|       |      |                                |      |                                                     |      |        |        |                      |
